# Recurrent Nerual Network
* 使用MNIST数据作为示例，该数据为手写的0~9数字的图像
* 尝试使用RNN对图像数字进行分类
* 教程[aymericdamien](https://github.com/aymericdamien/TensorFlow-Examples)方法基于tf.contrib，该模块在TensorFlow2.0中将被弃用
* tf.nn.rnn_cell与tf.contirb.rnn两个模块提供的RNN是相同的
* tf.nn.dynamic_rnn() tf.nn.bidirectional_dynamic_rnn() 可方便地提取hidden layers的信息
* tf.nn.static_rnn() 用法与上面的函数不同，教程里用的是该方法

## 示例
* 基于教程教程[aymericdamien](https://github.com/aymericdamien/TensorFlow-Examples)
* 修改部分方法，tf.contrib模块下的函数改为tf.nn模块下的函数

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# training parameters 训练用参数
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# network parameters 网络结构参数
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf graph inputs
X = tf.placeholder(tf.float32, [None, timesteps, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [6]:
def rnn_construct_static(input_x, weights, biases, hidden_size):
    input_x = tf.unstack(input_x, axis = 1)
    
    lstm_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
    outputs, _ = tf.nn.static_rnn(lstm_cell, input_x, dtype = tf.float32)
    
    return tf.nn.xw_plus_b(outputs[-1], weights, biases)

In [12]:
def rnn_construct_dynamic(input_x, weights, biases, hidden_size):
    lstm_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
    _, final_state = tf.nn.dynamic_rnn(lstm_cell, input_x, dtype = tf.float32)
    
    return tf.nn.xw_plus_b(final_state, weights, biases)

In [9]:
with tf.variable_scope("rnn_network_1"):
    # Define weights
    weights = tf.Variable(tf.random_normal([num_hidden, num_classes]))
    biases = tf.Variable(tf.random_normal([num_classes]))
    # Define network outputs
    logits = rnn_construct_static(X, weights, biases, num_hidden)
    prediction = tf.nn.softmax(logits)

In [10]:
with tf.variable_scope("training_1"):
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))  # prediction -> logits OK!
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.7392, Training Accuracy= 0.125
Step 200, Minibatch Loss= 2.1329, Training Accuracy= 0.289
Step 400, Minibatch Loss= 1.9307, Training Accuracy= 0.359
Step 600, Minibatch Loss= 1.7596, Training Accuracy= 0.477
Step 800, Minibatch Loss= 1.6280, Training Accuracy= 0.469
Step 1000, Minibatch Loss= 1.6127, Training Accuracy= 0.461
Step 1200, Minibatch Loss= 1.4764, Training Accuracy= 0.508
Step 1400, Minibatch Loss= 1.4134, Training Accuracy= 0.555
Step 1600, Minibatch Loss= 1.2770, Training Accuracy= 0.578
Step 1800, Minibatch Loss= 1.1860, Training Accuracy= 0.625
Step 2000, Minibatch Loss= 0.9845, Training Accuracy= 0.711
Step 2200, Minibatch Loss= 1.4661, Training Accuracy= 0.500
Step 2400, Minibatch Loss= 1.1462, Training Accuracy= 0.609
Step 2600, Minibatch Loss= 1.1971, Training Accuracy= 0.633
Step 2800, Minibatch Loss= 1.1023, Training Accuracy= 0.648
Step 3000, Minibatch Loss= 0.9388, Training Accuracy= 0.711
Step 3200, Minibatch Loss= 1.0490, Training Acc

## tf方法说明
大部分摘自官方文档。

### 类 [tf.nn.rnn_cell.LSTMCell](https://tensorflow.google.cn/api_docs/python/tf/nn/rnn_cell)
* 为模块 [tf.nn.rnn_cell](https://tensorflow.google.cn/api_docs/python/tf/nn/rnn_cell) 下的类，以本类LSTMCell为例讲解RNN的Cell
* **\_\_init\_\_(num_units, ...)** 有其他inputs，先不用管。
  * num_units: int, The number of units in the LSTM cell.
  * num_units == hidden layer dimension 即num_units设置的是RNN中隐藏层的维数

### 函数 [tf.nn.static_rnn](https://tensorflow.google.cn/api_docs/python/tf/nn/static_rnn)
* 输入: A length T list of inputs, each a Tensor of shape **[batch_size, input_size]**, or a nested tuple of such elements.
* 输出: A pair (outputs, state) where:
  * outputs is **a length T list** of outputs (one for each input), or a nested tuple of such elements.
  * state is the final state
* 说明：输出的特征维数同隐藏层维数，具体见下面函数的说明。

### 函数[tf.nn.dynamic_rnn](https://tensorflow.google.cn/api_docs/python/tf/nn/dynamic_rnn)
**tf.nn.dynamic_rnn(cell, inputs, ...)**

* 网上说该函数比上面函数具有更好的特性，下面提供一些网上的说法：
  * rnn在计算变长文本的时候，使用dynamic_rnn更加节省计算量，推荐使用这个。
  * 总的来说，优先使用dynamic_rnn，因为它的graph创建速度快，且可用于处理可变大小的batch。
* 输入: The RNN inputs. 
  * If time_major == False (default), this must be a Tensor of shape: **[batch_size, max_time, ...]**, or a nested tuple of such elements. 
  * If time_major == True, this must be a Tensor of shape: **[max_time, batch_size, ...]**, or a nested tuple of such elements.
* 输出: A pair (outputs, state) where:
  * outputs: The RNN output Tensor.
    * If time_major == False (default), this will be a Tensor shaped: **[batch_size, max_time, cell.output_size]**.  
    * If time_major == True, this will be a Tensor shaped: **[max_time, batch_size, cell.output_size]**.  
    * Note, if cell.output_size is a (possibly nested) tuple of integers or TensorShape objects, then outputs will be a tuple having the same structure as cell.output_size, containing Tensors having shapes corresponding to the shape data in cell.output_size.
  * state: The final state. If cell.state_size is an int, this will be shaped **[batch_size, cell.state_size]**. If it is a TensorShape, this will be shaped [batch_size] + cell.state_size. If it is a (possibly nested) tuple of ints or TensorShape, this will be a tuple having the corresponding shapes. If cells are LSTMCells state will be a tuple containing a LSTMStateTuple for each cell.
* 说明：一般来说，若建立rnn cell时未进行num_units外的设置，函数tf.nn.dynamic_rnn()输出中 **cell.output_size == cell.state_size == num_units**，即为隐藏层维数。

###  函数 [tf.unstack](https://tensorflow.google.cn/api_docs/python/tf/unstack)
**tf.unstack(value, num=None, axis=0, name='unstack')**

在使用tf.nn.static_rnn时会用到。

For example, given a tensor of shape **(A, B, C, D)**; 

If axis == 0 then the i'th tensor in output is the slice value[i, :, :, :] and each tensor in output will have shape **(B, C, D)**. (Note that the dimension unpacked along is gone, unlike split). 

If axis == 1 then the i'th tensor in output is the slice value[:, i, :, :] and each tensor in output will have shape **(A, C, D)**. Etc.